In [0]:
%python
# dbutils.fs.rm("/FileStore/tables/sales.csv")

Out[3]: True

In [0]:
%python
# dbutils.fs.ls("/FileStore/tables/")

In [0]:
%sql
-- DROP TABLE IF EXISTS sales;

-- CREATE TABLE sales USING CSV OPTIONS (path "/FileStore/tables/sales.csv", header "true")

In [0]:
%sql
select *
from sales
limit 10

date,id_client,id_product,sale_amount
01.01.2021,4,4,24
02.01.2021,1,1,1
03.01.2021,2,1,10
04.01.2021,10,3,80
05.01.2021,8,1,72
06.01.2021,9,3,72
07.01.2021,4,4,16
08.01.2021,5,3,30
09.01.2021,1,3,2
10.01.2021,8,5,48


In [0]:
%sql
drop table if exists report_abc;

create table report_abc (id_client int, client_amount int, percentage float, percentage_acc float, group_client string);

with tbl1 (select s.id_client,
                  sum(s.sale_amount) as client_amount
           from sales as s
           group by s.id_client
           order by sum(s.sale_amount) desc),

     tbl2 (select sum(t.client_amount) as total_amount  from tbl1 as t), 

     tbl3 (select t1.id_client, 
                  t1.client_amount, 
                  round(t1.client_amount / t2.total_amount,2) as percentage
           from tbl1 as t1 cross join tbl2 as t2
           order by t1.client_amount desc),
           
     tbl4 (select t.*, 
                  round(sum(t.percentage) over (order by t.client_amount desc rows between unbounded preceding and current row),2) as percentage_acc
           from tbl3 as t),
           
     tbl5 (select t.*,
                  case
                      when (t.percentage_acc>=0) and(t.percentage_acc<0.8) then "A"
                      when (t.percentage_acc>=0.8) and(t.percentage_acc<0.95) then "B"
                      else "C"
                  end as group_client
           from tbl4 as t)

insert into report_abc
select t.id_client, t.client_amount, t.percentage, t.percentage_acc, t.group_client
from tbl5 as t;

num_affected_rows,num_inserted_rows
10,10


In [0]:
%sql
select *
from report_abc

id_client,client_amount,percentage,percentage_acc,group_client
9,1197,0.2,0.2,A
8,976,0.17,0.37,A
10,740,0.13,0.5,A
6,738,0.13,0.63,A
7,714,0.12,0.75,A
5,485,0.08,0.83,B
4,360,0.06,0.89,B
3,348,0.06,0.95,C
2,246,0.04,0.99,C
1,41,0.01,1.0,C
